<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Интегрирование Верле

### Modules - Ordinary Differential Equations
<section class="post-meta">
By Magnus A. Gjennestad, Vegard Hagen, Aksel Kvaal, Morten Vassvik, Trygve B. Wiig and Peter Berg
</section>
Last edited: March 11th 2018 
___

Можем ли мы решить ОДУ второго порядка (например, из второго закона Ньютона) 
\begin{equation}
m\frac{d^2}{dt^2}x(t)=F(x(t),t),
\label{eq:1} \quad (1)
\end{equation}
не прибегая к методу Эйлера, который переформулирует проблему в два связанных ОДУ первого порядка?

Здесь мы используем начальное условие $x(t_0)=x_0$ и $v(t_0)=v_0$ и ограничиваем наш анализ одним измерением. 
Кроме того, на данный момент мы предполагаем, что сила $F$ не зависит от скорости $v$.

Мы можем разделить уравнение (1) на массу $m$ и написать вместо этого
$$
\frac{d^2}{dt^2}x(t)=a(x(t),t),
$$
где $a(x,t)$ - ускорение.

__Пример 1:__
\begin{equation}
\frac{d^2x}{dt^2}=-x+x^3+0.1\cos(t),
\label{eq:2} \quad (2)
\end{equation}
с начальными условиями $x(t_0)=0$ и $v(t_0)=0$.

Довольно грубым способом мы теперь приближаем вторую производную $x(t)$ по времени, т.е. ускорение, как
\begin{equation}
\begin{aligned}
\frac{d^2x}{dt^2}\bigg|_{t=t_n}=\frac{dv}{dt}\bigg|_{t=t_n}&
\approx&\frac{v(t_n+h/2)-v(t_n-h/2)}{h} \\
&\approx&\frac{
 \frac{x(t_n+h)-x(t_n)}{h}
-\frac{x(t_n)-x(t_n-h)}{h}
}{h},
\end{aligned}
\label{eq:3} \quad (3)
\end{equation}
где мы использовали __приближение центральной разности__
$$
\frac{dv}{dt}\bigg|_{t=t_n}\approx\frac{v(t_n+h/2)-v(t_n-h/2)}{h}
$$
где
$$v(t_n+h/2)\approx \frac{x(t_n+h)-x(t_n)}{h},\\
v(t_n-h/2)\approx \frac{x(t_n)-x(t_n-h)}{h}.$$


Приведенная выше формулировка следует обычной дискретизации
$$
t_n=t_0+n\cdot h~~~\mathrm{with}~~~n=0,1,2,3,...,N.
$$
Опять же, предполагается следующее обозначение: $x_n=x(t_n)$.

Имея это в виду, подстановка выражения (3) в уравнение (2)
и последующее умножение на $h^2$ дает
$$
x_{n+1}-2x_n+x_{n-1}=h^2\left[-x_n+x_n^3+0.1\cos(t_n)\right],
$$
где мы оценили уравнение (2) при $t=t_n$. Это приводит к следующей
рекурсивной формулировке решения
\begin{equation}
x_{n+1}=2x_n-x_{n-1}+h^2\left[-x_n+x_n^3+0.1\cos(t_n)\right]
\label{eq:4} \quad (4)
\end{equation}
with $x_0=0$ and $v_0=0$.

### Интегрирование Верле

Более строго можно вывести обобщенную версию рекурсивной формулы (4), а именно 

\begin{equation}
\boxed{x_{n+1}=2x_n-x_{n-1}+h^2 a(x_n,t_n),}
\label{eq:5} \quad (5)
\end{equation}

с помощью разложений Тейлора в окрестности $x(t_n)$ (см. Приложение). 
Метод разложения Тейлора говорит нам, что локальная ошибка, которую мы допускаем при использовании аппроксимации (5), масштабируется как $h^4$. Это, пожалуй, удивительно хорошо.

Метод (5) называется __интегрированием Верле__.
Часто он включает в себя ускорение, которое явно не зависит от времени:
$a=a(x(t))$. 

Оставшаяся проблема теперь состоит в том, чтобы использовать уравнение (4) при $t=t_0$, т.е. при $n=0$, поскольку мы не знаем значения $x$ для $t<0$, в частности $x_{-1}$.
Здесь можно оценить первое значение $x_1$ после начальной точки $t_0$ путем разложения в ряд $x(t)$ в $t=t_0$:
$$
x_1=x_0+h v_0 + \frac{h^2}{2}a(x_0,t_0)+\mathcal{O}(h^3).
$$
В нашем примере получается ($x_0=0$, $v_0=0$, $t_0=0$)
\begin{equation}
x_1=x_0+h v_0 + \frac{h^2}{2}a(x_0,t_0)=\frac{h^2}{20}.
\label{eq:6} \quad (6)
\end{equation}
После этого мы можем придерживаться соотношения (4), чтобы найти все остальные $x_n$.

(<u>Примечание</u>: Погрешность аппроксимации (6) масштабируется следующим образом $h^3$.)

Давайте использовать (4) и (6) для решения (2):

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Set common figure parameters

newparams = {'figure.figsize': (16, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.linewidth': 2,
             'font.size': 14}
plt.rcParams.update(newparams)


In [ ]:
N = 100000 # количество шагов
h = 0.001  # размер шага

t = np.zeros(N+1)
x = np.zeros(N+1)

# начальные значения
t_0 = 0
x_0 = 0
t[0] = t_0
x[0] = x_0

for n in range(N):
    # Вторая точка сетки:
    if n==0:
        t[n+1] = h
        x[n+1] = h**2/20.0
    # Интегрирование Верле
    else:
        t[n+1] = t[n] + h
        x[n+1] = 2.0*x[n] - x[n-1] + h**2*(-x[n]+x[n]**3+0.1*np.cos(t[n]))

# Построит график решения
plt.plot(t,x)
plt.ylabel(r'$x(t)$')
plt.xlabel(r'$t$')
plt.grid();

Здесь мы выбрали $h=0.001$ и $N=100\,000$,
и поэтому $t_N=100$.
На графике $x(t)$ дискретные точки были соединены
прямыми линиями.

#### Вопрос

Что, если сила и, следовательно, ускорение также зависят от скорости $v$?
Другими словами, давайте посмотрим на
$$
\frac{d^2}{dt^2}x(t)=a(x(t),v(t),t),
$$
с начальными условиями $x(t_0)=x_0$ и $v(t_0)=v_0$.

Мы приближаем левую сторону к $t=t_n$, как и раньше, и приближаем правую сторону к
$$
a(x_n,v_n,t_n)\approx a\left( x_n, \frac{x_{n+1}-x_{n-1}}{2h}, t_n \right),
$$
где мы снова использовали приближение центральной разности для $v_n$ :
$$
\frac{dv}{dt}\bigg|_{t=t_n}\approx\frac{v(t_n+h)-v(t_n-h)}{2h}.
\label{eq:7} \quad (7)
$$

Однако ошибка аппроксимации (7) масштабируется как $h^2$, тем самым снижая общую точность метода.

Опять же, мы оцениваем
$$
x_1=x_0+h v_0 + \frac{h^2}{2}a(x_0,v_0,t_0).
$$
После этого мы теперь используем
$$
x_{n+1}=2x_n-x_{n-1}+h^2 a\left( x_n, \frac{x_{n+1}-x_{n-1}}{2h}, t_n \right) .
$$
Обратите внимание, что нам нужно решить это уравнение для $x_{n+1}$, что не всегда возможно в аналитической форме. 

Давайте сосредоточимся на задаче, которая является линейной по $v$.

__Пример 2__:
$$
\frac{d^2x}{dt^2}=-v-x^3
$$
с начальными условиями $x_0=x(0)=10$ и $v_0=v(0)=0$. 
Используя наш приведенный выше формализм, уравнение читается в дискретизированной форме
\begin{align*}
x_{n+1}&=2x_n-x_{n-1}+h^2 \left( -\frac{x_{n+1}-x_{n-1}}{2h}-x_n^3 \right) \\
\Rightarrow x_{n+1}&=\frac{2x_n-(1-h/2)\,x_{n-1}-h^2 x_n^3}{1+h/2}.
\end{align*}
В сочетании с $x_0=10$ и
$$
x_1=x_0+h v_0 + \frac{h^2}{2}a(x_0,v_0,t_0)=10-500 h^2,
$$
получаем полностью формализованную задачу. Она реализована в приведенном ниже коде. В коде мы выбираем $h=0.001$ и 
$N=3\,000$, и поэтому $t_N=3.0$

In [ ]:
N = 3000 # количество шагов
h = 0.001 # размер шага

t = np.zeros(N+1)
x = np.zeros(N+1)

# начальные значения
t_0 = 0
x_0 = 10
t[0] = t_0
x[0] = x_0

for n in range(N):
    # Вторая точка сетки:
    if n==0:
        t[n+1] = h
        x[n+1]= 10.0-500.0*h**2
    # Интегрирование Верле
    else:
        t[n+1] = t[n] + h
        x[n+1] = (2.0*x[n]-(1.0-h/2)*x[n-1]-h**2*x[n]**3)/(1+h/2.0)

        
# Построит график решения
plt.plot(t,x)
plt.ylabel(r'$x(t)$')
plt.xlabel(r'$t$')
plt.grid();

### Выводы

Мы видели, что существует несколько методов для решения ОДУ.
Интегрирование Верле часто используется для решения уравнений из второго закона Ньютона. 
1. Оно обеспечивает более высокую точность, чем метод Эйлера.
2. Оно более стабильно, чем метод Эйлера, и
3. может осилить нелинейное решение для $x_{n+1}$. 
4. Его можно легко обобщить на трехмерное движение.

### Приложение: Вывод формулы Верле используя ряды Тейлора

Учитывая динамику $x(t)$ при $t=t_n$, давайте аппроксимируем значения 
$x_{n-1}=x(t_n-h)$ и $x_{n+1}=x(t_n+h)$ с помощью рядов Тейлора 
at $x_{n}=x(t_n)$:
\begin{eqnarray*}
x(t_n-h)&=&x(t_n)+v(t_n) \cdot (-h)+\frac{a(t_n)}{2}\cdot (-h)^2+\frac{b(t_n)}{6}\cdot 
(-h)^3+\mathcal{O}(h^4), \\ 
x(t_n+h)&=&x(t_n)+v(t_n)\,h+\frac{a(t_n)}{2}h^2+\frac{b(t_n)}{6}h^3+\mathcal{O}(h^4),
\end{eqnarray*}
где 
\begin{align*}
v(t_n)=\frac{dx}{dt}\bigg|_{t=t_n}, \quad 
a(t_n)=\frac{d^2x}{dt^2}\bigg|_{t=t_n}, \quad \text{and} \quad
b(t_n)=\frac{d^3x}{dt^3}\bigg|_{t=t_n}.
\end{align*}
Сложение этих уравнений и последующая перестановка слагаемых дает 
$$ 
x(t_n+h)=2x(t_n)-x(t_n-h)+a(t_n)\,h^2+\mathcal{O}(h^4)
$$
или, в другой нотации,
$$
x_{n+1}=2 x_n-x_{n-1}+h^2\,a(t_n)+\mathcal{O}(h^4).
$$
Это наша формула Верле (5), если мы отбросим слагаемые более высокого порядка $\mathcal{O}(h^4)$
и напишем $a(x(t_n), t_n)=a(x_n, t_n)$ вместо просто $a(t_n)$.